In [87]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

In [88]:
a = pd.read_excel('./baseball_1.xlsx')
print(a)

    순위    AVG   G   PA   AB   R   H  2B  3B  HR   TB  RBI  SAC  SF
0    1  0.390  46  195  177  31  69  10   0   7  100   38    0   3
1    2  0.389  45  187  162  33  63  12   0   2   81   27    0   4
2    3  0.361  47  214  194  32  70  15   2   7  110   27    0   1
3    4  0.340  43  192  162  19  55   8   0   4   75   21    0   0
4    5  0.332  49  223  208  38  69  12   0  14  123   46    0   2
5    6  0.331  44  179  160  20  53   6   0   6   77   41    0   5
6    7  0.328  46  203  186  39  61   6   2  11  104   30    1   2
7    8  0.324  47  196  173  35  56   8   0   7   85   31    0   0
8    9  0.323  46  191  167  30  54  12   1   3   77   25    0   1
9   10  0.322  46  200  180  26  58  10   1   6   88   32    0   4
10  11  0.319  48  220  191  37  61  13   0  14  116   38    0   2
11  12  0.319  51  222  207  32  66  13   5  10  119   38    1   3
12  13  0.314  40  176  159  20  50  11   1   7   84   31    0   1
13  14  0.313  41  184  166  30  52   6   1   7   81   29    0

In [89]:
x_train = a.drop(columns=['순위'], axis = 1).values
y_train = a[['순위']].values

In [90]:
x_train

array([[  0.39 ,  46.   , 195.   , 177.   ,  31.   ,  69.   ,  10.   ,
          0.   ,   7.   , 100.   ,  38.   ,   0.   ,   3.   ],
       [  0.389,  45.   , 187.   , 162.   ,  33.   ,  63.   ,  12.   ,
          0.   ,   2.   ,  81.   ,  27.   ,   0.   ,   4.   ],
       [  0.361,  47.   , 214.   , 194.   ,  32.   ,  70.   ,  15.   ,
          2.   ,   7.   , 110.   ,  27.   ,   0.   ,   1.   ],
       [  0.34 ,  43.   , 192.   , 162.   ,  19.   ,  55.   ,   8.   ,
          0.   ,   4.   ,  75.   ,  21.   ,   0.   ,   0.   ],
       [  0.332,  49.   , 223.   , 208.   ,  38.   ,  69.   ,  12.   ,
          0.   ,  14.   , 123.   ,  46.   ,   0.   ,   2.   ],
       [  0.331,  44.   , 179.   , 160.   ,  20.   ,  53.   ,   6.   ,
          0.   ,   6.   ,  77.   ,  41.   ,   0.   ,   5.   ],
       [  0.328,  46.   , 203.   , 186.   ,  39.   ,  61.   ,   6.   ,
          2.   ,  11.   , 104.   ,  30.   ,   1.   ,   2.   ],
       [  0.324,  47.   , 196.   , 173.   ,  35.   ,  56.   , 

In [91]:
print(x_train.shape)
print(y_train.shape)

(30, 13)
(30, 1)


In [92]:
print(np.unique(y_train))
print(len(np.unique(y_train)))

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 20 21 22 23 25 26
 27 28 29 30]
28


In [93]:
from torch.utils.data import DataLoader, Dataset, TensorDataset

class custom_data(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]

        x = torch.tensor(x, dtype = torch.float32)
        y = torch.tensor(y, dtype = torch.long)

        return x, y

In [94]:
train_dataset = custom_data(x_train, y_train)
train_loader = DataLoader(train_dataset)

In [95]:
x_in, y_out = next(iter(train_loader))

print(x_in.shape)
print(y_out.shape)

print(x_in.dtype)
print(y_out.dtype)

torch.Size([1, 13])
torch.Size([1, 1])
torch.float32
torch.int64


In [100]:
import torch.nn.functional as F

class Mymodel(nn.Module):
    def __init__(self):
        super(Mymodel, self).__init__()
        self.fc1 = nn.Linear(13, 50)
        self.fc2 = nn.Linear(50, 40)
        self.fc3 = nn.Linear(40, 31)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)

        out = self.fc3(x)
        return out
    
model = Mymodel()
print(model)

Mymodel(
  (fc1): Linear(in_features=13, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=40, bias=True)
  (fc3): Linear(in_features=40, out_features=31, bias=True)
)


In [97]:
from torchsummary import summary
# 입력의 크기를 넣어줘야 함
summary(model, (1, 13), device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 50]             700
            Linear-2                [-1, 1, 40]           2,040
            Linear-3                [-1, 1, 31]           1,271
Total params: 4,011
Trainable params: 4,011
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.02
Estimated Total Size (MB): 0.02
----------------------------------------------------------------


In [101]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

num_epochs = 850

for epoch in range(num_epochs):
    for x_batch, y_batch in train_loader:
        outputs = model(x_batch)

        optimizer.zero_grad()
        loss = criterion(outputs, y_batch.squeeze(dim=-1))
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch {epoch+1}, Loss : {loss.item()}')

Epoch 100, Loss : 2.381150245666504
Epoch 200, Loss : 1.3861006498336792
Epoch 300, Loss : 1.297465205192566
Epoch 400, Loss : 0.9910264015197754
Epoch 500, Loss : 0.048543039709329605
Epoch 600, Loss : 0.03720228374004364
Epoch 700, Loss : 0.03915843367576599
Epoch 800, Loss : 0.006207234691828489


In [107]:
b = pd.read_excel('./baseball_1.xlsx')
print(b)

    순위    AVG   G   PA   AB   R   H  2B  3B  HR   TB  RBI  SAC  SF
0    1  0.390  46  195  177  31  69  10   0   7  100   38    0   3
1    2  0.389  45  187  162  33  63  12   0   2   81   27    0   4
2    3  0.361  47  214  194  32  70  15   2   7  110   27    0   1
3    4  0.340  43  192  162  19  55   8   0   4   75   21    0   0
4    5  0.332  49  223  208  38  69  12   0  14  123   46    0   2
5    6  0.331  44  179  160  20  53   6   0   6   77   41    0   5
6    7  0.328  46  203  186  39  61   6   2  11  104   30    1   2
7    8  0.324  47  196  173  35  56   8   0   7   85   31    0   0
8    9  0.323  46  191  167  30  54  12   1   3   77   25    0   1
9   10  0.322  46  200  180  26  58  10   1   6   88   32    0   4
10  11  0.319  48  220  191  37  61  13   0  14  116   38    0   2
11  12  0.319  51  222  207  32  66  13   5  10  119   38    1   3
12  13  0.314  40  176  159  20  50  11   1   7   84   31    0   1
13  14  0.313  41  184  166  30  52   6   1   7   81   29    0

In [108]:
x_test = b.drop(columns=['순위'], axis = 1).values
y_test = b[['순위']].values

In [109]:
print(x_test)
print(y_test)

[[  0.39   46.    195.    177.     31.     69.     10.      0.      7.
  100.     38.      0.      3.   ]
 [  0.389  45.    187.    162.     33.     63.     12.      0.      2.
   81.     27.      0.      4.   ]
 [  0.361  47.    214.    194.     32.     70.     15.      2.      7.
  110.     27.      0.      1.   ]
 [  0.34   43.    192.    162.     19.     55.      8.      0.      4.
   75.     21.      0.      0.   ]
 [  0.332  49.    223.    208.     38.     69.     12.      0.     14.
  123.     46.      0.      2.   ]
 [  0.331  44.    179.    160.     20.     53.      6.      0.      6.
   77.     41.      0.      5.   ]
 [  0.328  46.    203.    186.     39.     61.      6.      2.     11.
  104.     30.      1.      2.   ]
 [  0.324  47.    196.    173.     35.     56.      8.      0.      7.
   85.     31.      0.      0.   ]
 [  0.323  46.    191.    167.     30.     54.     12.      1.      3.
   77.     25.      0.      1.   ]
 [  0.322  46.    200.    180.     26.     58.

In [110]:
test_dataset = custom_data(x_test, y_test)
test_loader = DataLoader(test_dataset)

In [113]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for x_batch, y_batch in test_loader:
        outputs = model(x_batch)

        _, predicted = torch.max(outputs.data, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

print(f'Accuracy : {100 * correct / total}%, Pred : {_}')

Accuracy : 90.0%, Pred : tensor([17.6739])
